In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import math

In [2]:
import os
print(os.getcwd())
path = str(os.getcwd())
path = path.replace('\\economic_data\\notebooks', "")
path = path.replace('\\', "/")

c:\programowanie\bond\economic_data\notebooks


In [3]:
bonds = pd.read_csv(path + "/economic_data/bonds.csv", delimiter=";")
bonds

,bond_type,maturity,procent_first_year,how_often_change,index_pointer,margin,interest_payment,interest_capitalization,exchange_price_PLN,cost_of_early_buyout_PLN,unchanged_value_early_buyout_moths
0,ROR,12,0.0675,1,NBP interest rates,0.0000,1,-,99.8,0.5,1
1,DOR,24,0.0685,1,NBP interest rates,0.0010,1,-,99.7,0.7,1
2,TOS,36,0.0685,36,WIBOR6M,0.0000,-,12,99.6,0.7,maturity
3,COI,48,0.0700,12,inflation,0.0100,12,-,99.9,0.7,12
4,EDO,120,0.0725,12,inflation,0.0125,-,12,99.4,2.0,maturity
5,ROS,72,0.0720,12,inflation,0.0150,-,12,-,0.7,maturity
6,ROD,144,0.0750,12,inflation,0.0175,-,12,-,2.0,maturity


In [4]:
inflation_df = pd.read_csv(path + "/economic_data/predicted_inflation.csv")
inflation_df.head()

,inflation,upper,lower,month,year
0,9.82,11.05,8.60,8,2023
1,8.72,10.70,6.74,9,2023
2,7.59,10.19,5.00,10,2023
3,6.48,9.65,3.31,11,2023
4,5.41,9.14,1.68,12,2023


In [5]:
def get_maturity_list(bond, period):
    list_maturity = []

    for i in range(0, period):
        if ((i + 1) % bond["maturity"].values.astype(int)[0]) == 0:
            list_maturity.append("yes")
        else:
            list_maturity.append("no")
    
    return list_maturity

def get_intrest_list(bond, period, interest_list, list_maturity):
    list_intrest = []
    interest_index = 0    

    new_month = 0
    for i in range(0, period):
        if i % 12 == 0:
            interest = interest_list[interest_index]
            interest_index += 1
        
        if interest_index == 1:
            list_intrest.append(round(interest, 4))
        
        elif list_maturity[i - 1] == "yes" or new_month != 0:
            list_intrest.append(round(interest, 4))
            new_month += 1
            
            if new_month == 12:
                new_month = 0
            
        else:
            list_intrest.append(round(bond["margin"].values.astype(float)[0] + interest, 4))
            
    return list_intrest

def get_gross_value_list(bond, period, base_value_list, interest_list):
    
    try:
        interest_payment = bond["interest_payment"].values.astype(int)[0]
    except ValueError:
        interest_payment = bond["maturity"].values.astype(int)[0]
    
    list_gross = []
    for i in range(0, period):
        if (i + 1) % interest_payment != 0:
            ifPayment = (i + 1) % interest_payment
        else:
            ifPayment = interest_payment
        
        list_gross.append( round(base_value_list[i] * (1 + interest_list[i] * ifPayment / 12), 2))
    
    return list_gross

def get_cost_of_early_buyout(bond, period, base_value_list, list_gross, bont_count_list = ""):
    maturity = bond["maturity"].values.astype(int)[0]
    
    try:
        unchanged_value_early_buyout_moths = bond["unchanged_value_early_buyout_moths"].values.astype(int)[0]
    except ValueError:
        unchanged_value_early_buyout_moths = maturity
    
    cost_of_early_buyout_list = []
    
    if not bont_count_list:
        bont_count_list = [base_value_list[0] / 100] * period
    
    for i in range(0, period):
        if (i + 1) % maturity == 0:
            cost_of_early_buyout_list.append(0)
            
        elif ((i + 1) % maturity < maturity) and ((i + 1) % maturity <= unchanged_value_early_buyout_moths):
            cost_of_early_buyout_list.append(round(min(bond["cost_of_early_buyout_PLN"].values.astype(float)[0] * bont_count_list[i], list_gross[i] - base_value_list[i]), 2))
            
        else:
            cost_of_early_buyout_list.append(round(bond["cost_of_early_buyout_PLN"].values.astype(float)[0] * bont_count_list[i], 2))
            
    return cost_of_early_buyout_list



def get_paycheck(bond, period, base_value_list, list_gross, list_maturity, end_moth_buyout, tax):    
    try:
        interest_payment = bond["interest_payment"].values.astype(int)[0]
    except ValueError:
        return [0] * period
    
    nominal_value = base_value_list
    balance_list = [0]
    
    for i in range(0, period):
        if list_maturity[i] == "yes" and i > 0:
            balance_list.append(round(end_moth_buyout[i] - math.floor(end_moth_buyout[i]  / bond["exchange_price_PLN"].values.astype(float)[0]) * bond["exchange_price_PLN"].values.astype(float)[0], 2))
        elif (i + 1) % interest_payment == 0:
            balance_list.append(round((list_gross[i] - nominal_value[i]) * (1 - tax), 2))
        else:
            balance_list.append(0)
        
    balance_list.pop(0)
    return balance_list

def get_bond_count(bond, period, cost, list_maturity, end_moth_buyout, balance):
    try:
        exchange = bond["exchange_price_PLN"].values.astype(float)[0]
    except ValueError:
        exchange = 100.0
    
    bond_count_list = [cost / 100]
    list_maturity = list_maturity
    
    for i in range(0, period):
        if list_maturity[i] == "yes" and i > 0:
            bond_count_list.append(round(math.floor(end_moth_buyout[i] / exchange) + math.floor( (balance[i]) / 100), 2))

        else:
            bond_count_list.append(bond_count_list[-1])
    
    bond_count_list.pop(0)
    return bond_count_list


def get_total(bond, period, balance_list, end_moth_buyout):
    total = []
    balance_list = [0.0] + balance_list
    maturity = bond["maturity"].values.astype(int)[0]
    
    for i in range(0, period):
        ifmod = 0
        if i % maturity == 0:
            ifmod = math.floor(balance_list[i -1] / 100) * 100

        total.append(round(balance_list[i -1] - ifmod + end_moth_buyout[i],2))

    balance_list.pop(0)
    return total


def get_balance(period, paycheck, list_maturity):
    balance_list = [0]
    
    for i in range(0, period):
        ifyes = 0
        if list_maturity[i - 1] == "yes" and i > 0:
            ifyes = math.floor(balance_list[-1] / 100) * 100
        
        balance_list.append(round(balance_list[-1] - ifyes  + paycheck[i], 2))
        
    balance_list.pop(0)
    return balance_list

def get_gross_value_list(bond, period, base_value_list, interest_list):
    interest_capitalization = -1
    try:
        interest_payment = bond["interest_payment"].values.astype(int)[0]
    except ValueError:
        interest_payment = bond["maturity"].values.astype(int)[0]
        interest_capitalization	= bond["interest_capitalization"].values.astype(int)[0]
    
    list_gross = []
    for i in range(0, period):
        if (i + 1) % interest_payment != 0 and interest_capitalization == -1:
            ifPayment = (i + 1) % interest_payment
        elif interest_capitalization == -1:
            ifPayment = interest_payment
        elif (i + 1) % interest_capitalization != 0:
            ifPayment = (i + 1) % interest_capitalization
        else:
            ifPayment = 12
        
        list_gross.append( round(base_value_list[i] * (1 + interest_list[i] * ifPayment / 12), 2))
    
    return list_gross


def get_nominalValue(bond, period, list_maturity, bond_count_list, cost, gross_list):
    try:
        interest_capitalization	= bond["interest_capitalization"].values.astype(int)[0]
    except ValueError:
        interest_capitalization = -1
        
    nominal_list = [cost]
    list_maturity = ["no"] + list_maturity
    
    for i in range(0, period):
        if list_maturity[i] == "yes":
            nominal_list.append(bond_count_list[i] * 100)
        else:
            nominal_list.append(nominal_list[-1])
            
        if (i+1) % interest_capitalization == 1 and i > 0 and interest_capitalization != -1:
            nominal_list[i + 1] = gross_list[i - 1]
        
    nominal_list.pop(0)
    
    return nominal_list


def get_end_moth_buyout(bond, period, base_value_list, list_gross, cost_of_early_buyout, tax):
    nominal_value = []
    end_moth_buyout_list = []
    maturity = bond["maturity"].values.astype(int)[0]
    
    try:
        interest_capitalization	= bond["interest_capitalization"].values.astype(int)[0]
        for i in range(0, period):
            if (i + 1) % maturity == 0 or i == 0:
                nominal_value.append(base_value_list[i])
            else:
                nominal_value.append(nominal_value[-1])
                
    except ValueError:
        nominal_value = base_value_list
    
    for i in range(0, period):
        if i == 0:
            end_moth_buyout_list.append(round(nominal_value[i], 2))
        else:
            temp = list_gross[i] - cost_of_early_buyout[i] - (list_gross[i] - nominal_value[i - 1] - cost_of_early_buyout[i]) * tax
            end_moth_buyout_list.append(round(temp, 2))
        
    return end_moth_buyout_list

In [6]:
def get_bond_df(bond, period, predicted_intrest, cost, tax = 0.19):
    base_value = [cost] * period

    maturity_list = get_maturity_list(bond, period)
    intrest_list = get_intrest_list(bond, period, predicted_intrest, maturity_list)
    for i in range(0, math.ceil(144 / bond["maturity"].values.astype(int)[0]) + 12):
        gross_list = get_gross_value_list(bond, period, base_value, intrest_list)
        cost_of_early_buyout = get_cost_of_early_buyout(bond, period, base_value, gross_list)
        end_moth_buyout = get_end_moth_buyout(bond, period, base_value, gross_list, cost_of_early_buyout, tax)
        paycheck = get_paycheck(bond, period, base_value, gross_list, maturity_list, end_moth_buyout, tax)
        balance = get_balance(period, paycheck, maturity_list)
        total = get_total(bond, period, balance, end_moth_buyout)
        bond_count = get_bond_count(bond, period, cost, maturity_list, end_moth_buyout, balance)
        base_value = get_nominalValue(bond, period, maturity_list, bond_count, cost, gross_list)
        
        
    df = pd.DataFrame({"maturity" : maturity_list, "intrest" : intrest_list, "base_value" : base_value, "gross_list" : gross_list,
                        "cost_of_early_buyout" : cost_of_early_buyout, "end_moth_buyout" : end_moth_buyout, "paycheck" : paycheck, "balance" : balance,
                        "bond_count" : bond_count, "total" : total})

    return df


period = 144
cost = 50000


def get_inflation_models(period = 144, cost = 50000):
    bonds = pd.read_csv(path + "/economic_data/bonds.csv", delimiter=";")
    inflation_df = pd.read_csv(path + "/economic_data/predicted_inflation.csv")

    if datetime.now().month > 1:
        currentMonth = datetime.now().month - 1
    else:
        currentMonth = 12

    res = {}   
    for index, row in bonds[bonds["index_pointer"] == "inflation"].iterrows():
        temp = row.to_dict()
        bond = pd.DataFrame([temp]) 
        
        #inflation_list = inflation_df[inflation_df["month"] == currentMonth]
        inflation_list = [bond["procent_first_year"].values.astype(float)[0]] + [round(x / 100, 4) for x in inflation_df["inflation"].to_list()]
        print(inflation_list)
        bond_df = get_bond_df(bond, period, inflation_list, cost, 0.19)
        res[bond["bond_type"].values.astype(str)[0]] = bond_df
    
    return res
    
pop = get_inflation_models(144, 50000)

[0.07, 0.0982, 0.0872, 0.0759, 0.0648, 0.0541, 0.0441, 0.0351, 0.0273, 0.021, 0.0161, 0.0128, 0.0111, 0.011, 0.0124, 0.0151, 0.019, 0.0238, 0.0295, 0.0356, 0.042, 0.0485, 0.0548, 0.0606, 0.0658, 0.0702, 0.0737, 0.0761, 0.0774, 0.0776, 0.0768, 0.0748, 0.0719, 0.068, 0.0635, 0.0584, 0.0528, 0.047, 0.0412, 0.0355, 0.03, 0.0249, 0.0204, 0.0166, 0.0135, 0.0111, 0.0096, 0.0089, 0.0091, 0.0099, 0.0114, 0.0135, 0.0161, 0.019, 0.0222, 0.0255, 0.0287, 0.0318, 0.0347, 0.0372, 0.0393, 0.0409, 0.042, 0.0424, 0.0423, 0.0417, 0.0405, 0.0388, 0.0367, 0.0342, 0.0315, 0.0286, 0.0255, 0.0225, 0.0195, 0.0168, 0.0142, 0.0119, 0.01, 0.0085, 0.0074, 0.0067, 0.0065, 0.0066, 0.0071, 0.008, 0.0091, 0.0104, 0.012, 0.0136, 0.0152, 0.0169, 0.0184, 0.0198, 0.021, 0.022, 0.0227, 0.0231, 0.0233, 0.0231, 0.0227, 0.022, 0.021, 0.0198, 0.0185, 0.0171, 0.0155, 0.0139, 0.0124, 0.0108, 0.0094, 0.0081, 0.007, 0.0061, 0.0053, 0.0048, 0.0045, 0.0044, 0.0045, 0.0048, 0.0053, 0.0059, 0.0066, 0.0074, 0.0082, 0.009, 0.0098, 0.010

In [7]:
pop["ROS"].tail()

,maturity,intrest,base_value,gross_list,cost_of_early_buyout,end_moth_buyout,paycheck,balance,bond_count,total
139,no,0.0278,101104.61,102978.42,350.0,97848.09,0,0,765.0,97848.09
140,no,0.0278,101104.61,103212.64,350.0,98037.80,0,0,765.0,98037.80
141,no,0.0278,101104.61,103446.87,350.0,98227.53,0,0,765.0,98227.53
142,no,0.0278,101104.61,103681.09,350.0,98417.25,0,0,765.0,98417.25
143,yes,0.0278,101104.61,103915.32,0.0,98890.48,0,0,988.0,98890.48


In [8]:
os.chdir(path)

In [9]:
import bondModel

inflation_df = pd.read_csv(path + "/economic_data/predicted_inflation.csv")

res, inflation_list = bondModel.get_inflation_models(inflation_df["inflation"].to_list(), 144, 50000)

[0.07, 0.0982, 0.0872, 0.0759, 0.0648, 0.0541, 0.0441, 0.0351, 0.0273, 0.021, 0.0161, 0.0128, 0.0111, 0.011, 0.0124, 0.0151, 0.019, 0.0238, 0.0295, 0.0356, 0.042, 0.0485, 0.0548, 0.0606, 0.0658, 0.0702, 0.0737, 0.0761, 0.0774, 0.0776, 0.0768, 0.0748, 0.0719, 0.068, 0.0635, 0.0584, 0.0528, 0.047, 0.0412, 0.0355, 0.03, 0.0249, 0.0204, 0.0166, 0.0135, 0.0111, 0.0096, 0.0089, 0.0091, 0.0099, 0.0114, 0.0135, 0.0161, 0.019, 0.0222, 0.0255, 0.0287, 0.0318, 0.0347, 0.0372, 0.0393, 0.0409, 0.042, 0.0424, 0.0423, 0.0417, 0.0405, 0.0388, 0.0367, 0.0342, 0.0315, 0.0286, 0.0255, 0.0225, 0.0195, 0.0168, 0.0142, 0.0119, 0.01, 0.0085, 0.0074, 0.0067, 0.0065, 0.0066, 0.0071, 0.008, 0.0091, 0.0104, 0.012, 0.0136, 0.0152, 0.0169, 0.0184, 0.0198, 0.021, 0.022, 0.0227, 0.0231, 0.0233, 0.0231, 0.0227, 0.022, 0.021, 0.0198, 0.0185, 0.0171, 0.0155, 0.0139, 0.0124, 0.0108, 0.0094, 0.0081, 0.007, 0.0061, 0.0053, 0.0048, 0.0045, 0.0044, 0.0045, 0.0048, 0.0053, 0.0059, 0.0066, 0.0074, 0.0082, 0.009, 0.0098, 0.010

In [10]:
res["ROS"].tail()

,maturity,intrest,base_value,gross_list,cost_of_early_buyout,end_moth_buyout,paycheck,balance,bond_count,total
139,no,0.0278,101104.61,102978.42,350.0,97848.09,0,0,765.0,97848.09
140,no,0.0278,101104.61,103212.64,350.0,98037.80,0,0,765.0,98037.80
141,no,0.0278,101104.61,103446.87,350.0,98227.53,0,0,765.0,98227.53
142,no,0.0278,101104.61,103681.09,350.0,98417.25,0,0,765.0,98417.25
143,yes,0.0278,101104.61,103915.32,0.0,98890.48,0,0,988.0,98890.48
